#Análise socioeconômica Enem 2020 (RJ)

Fonte de dados: https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Extração, transformação e carregamento dos dados (ETL)

* Seleção dos registros referentes aos estudantes do Rio de Janeiro (candidatos que realizaram provas no RJ);
* Remover 'Treinieros' (alunos que só se inscreveram para treinar);
* Seleção de colunas que serão utilizadas;
* Tratamento de valores; 
* Carregamento do resultado em um novo arquivo CSV. 

## Extração

In [2]:
# Extração do arquivo csv do zip
## Esse passo foi necessário devido o tamnho do arquivo, que impediu o upload normal para o GDrive
import zipfile
zip_file = zipfile.ZipFile('/content/drive/MyDrive/Projetos/Python/analise_socio_economica_enem_2020/data/microdados_enem_2020.zip', 'r')
csv_file = zip_file.extract('DADOS/MICRODADOS_ENEM_2020.csv', path = '/content/drive/MyDrive/Projetos/Python/analise_socio_economica_enem_2020/data')

In [3]:
import pandas as pd

original_dataset = pd.read_csv('/content/drive/MyDrive/Projetos/Python/analise_socio_economica_enem_2020/data/DADOS/MICRODADOS_ENEM_2020.csv', sep=';', encoding='iso-8859-1')

#Tamanho do conjunto de dados
print(f'O conjunto de dados original tem {original_dataset.shape[0]} registros, com {original_dataset.shape[1]} variáveis')

#Tipos de dados
print('Com os seguintes tipo de dados: ')
original_dataset.dtypes

O conjunto de dados original tem 5783109 registros, com 76 variáveis
Com os seguintes tipo de dados: 


NU_INSCRICAO        int64
NU_ANO              int64
TP_FAIXA_ETARIA     int64
TP_SEXO            object
TP_ESTADO_CIVIL     int64
                    ...  
Q021               object
Q022               object
Q023               object
Q024               object
Q025               object
Length: 76, dtype: object

In [4]:
original_dataset.head()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,200006271946,2020,11,F,1,2,1,1,11,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200001195856,2020,11,M,2,3,1,1,11,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200001943954,2020,4,F,2,3,2,2,0,2,...,B,A,A,B,A,A,A,A,A,A
3,200001908998,2020,2,M,1,3,1,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200001634757,2020,4,F,1,3,2,1,1,1,...,A,A,A,B,A,B,B,A,A,B


In [5]:
#Seleção de alunos que realizaram a prova no Rio de Janeiro
select_rj_data = original_dataset['SG_UF_PROVA'] == 'RJ'
rj_dataset = original_dataset[select_rj_data]

#Removendo 'Treineiros'
rj_dataset = rj_dataset.query('IN_TREINEIRO == 0')


rj_dataset.head()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
33,200001699039,2020,12,F,1,2,1,1,12,1,...,A,A,A,B,A,A,B,A,B,A
67,200001104094,2020,6,F,1,2,1,1,4,1,...,A,A,A,B,A,A,D,A,A,B
93,200002727241,2020,4,M,1,2,1,2,0,2,...,B,A,B,C,B,B,D,A,C,B
130,200002186164,2020,2,F,1,3,1,2,0,2,...,B,A,A,C,A,A,C,A,B,B
137,200003838320,2020,7,F,1,3,1,1,5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#Seleção das colunas
select_columns = [# Dados cadastrais:
                  'NU_INSCRICAO',
                  'TP_FAIXA_ETARIA',
                  'TP_SEXO',
                  'TP_ESTADO_CIVIL',
                  'TP_COR_RACA',
                  #'TP_NACIONALIDADE',
                  'TP_ST_CONCLUSAO',
                  'TP_ANO_CONCLUIU',
                  'TP_ESCOLA',
                  'TP_ENSINO',
                  # Dados de realização da prova:
                  'NO_MUNICIPIO_PROVA',
                  'SG_UF_PROVA',
                  'TP_PRESENCA_CN',
                  'TP_PRESENCA_CH',
                  'TP_PRESENCA_LC',
                  'TP_PRESENCA_MT',
                  # Notas:
                  'NU_NOTA_CN',
                  'NU_NOTA_CH',
                  'NU_NOTA_LC',
                  'NU_NOTA_MT',
                  'TP_LINGUA',
                  'NU_NOTA_REDACAO',
                  # Questionário socioeconômico, questões selecionadas:
                  'Q001',  # Escolaridade Pai
                  'Q002',  # Escolaridade Mãe
                  'Q005',  # Quantidade de pessoas com quem divide a casa
                  'Q006',  # Renda mensal bruta familiar
                  'Q022',  # Possui celular?
                  'Q024',  # Possui computador?
                  'Q025',] # Possui acesso a internet? 

rj_dataset = rj_dataset[select_columns]

print(f'O novo conjunto de dados possui {rj_dataset.shape[0]} linhas e {rj_dataset.shape[1]} colunas')
print('Com os seguintes tipos de dados: ')
rj_dataset.dtypes

O novo conjunto de dados possui 356206 linhas e 28 colunas
Com os seguintes tipos de dados: 


NU_INSCRICAO            int64
TP_FAIXA_ETARIA         int64
TP_SEXO                object
TP_ESTADO_CIVIL         int64
TP_COR_RACA             int64
TP_ST_CONCLUSAO         int64
TP_ANO_CONCLUIU         int64
TP_ESCOLA               int64
TP_ENSINO             float64
NO_MUNICIPIO_PROVA     object
SG_UF_PROVA            object
TP_PRESENCA_CN          int64
TP_PRESENCA_CH          int64
TP_PRESENCA_LC          int64
TP_PRESENCA_MT          int64
NU_NOTA_CN            float64
NU_NOTA_CH            float64
NU_NOTA_LC            float64
NU_NOTA_MT            float64
TP_LINGUA               int64
NU_NOTA_REDACAO       float64
Q001                   object
Q002                   object
Q005                  float64
Q006                   object
Q022                   object
Q024                   object
Q025                   object
dtype: object

In [7]:
rj_dataset.head()

,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,NO_MUNICIPIO_PROVA,...,NU_NOTA_MT,TP_LINGUA,NU_NOTA_REDACAO,Q001,Q002,Q005,Q006,Q022,Q024,Q025
33,200001699039,12,F,1,2,1,12,1,NaN,Campos dos Goytacazes,...,NaN,1,NaN,A,B,1.0,B,B,B,A
67,200001104094,6,F,1,2,1,4,1,NaN,Rio de Janeiro,...,NaN,0,NaN,B,E,6.0,B,D,A,B
93,200002727241,4,M,1,2,2,0,2,1.0,Itaguaí,...,743.0,0,920.0,E,E,4.0,F,D,C,B
130,200002186164,2,F,1,3,2,0,2,1.0,Rio de Janeiro,...,538.6,1,560.0,C,C,2.0,B,C,B,B
137,200003838320,7,F,1,3,1,5,1,NaN,Duque de Caxias,...,411.8,1,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Transformação

Com base no dicionário de dados (disponível no diretório \data):

* Corrigir possíveis problemas com os dados;
* Renomear colunas;
* Substituir códigos por valores descritivos.



In [8]:
# Renomeando colunas
rj_dataset.rename(columns={'NU_INSCRICAO': 'incricao',
                  'TP_FAIXA_ETARIA': 'faixa_etaria',
                  'TP_SEXO': 'sexo',
                  'TP_ESTADO_CIVIL': 'estado_civil',
                  'TP_COR_RACA': 'cor_raca',
                  #'TP_NACIONALIDADE': 'nacionalidade',
                  'TP_ST_CONCLUSAO': 'status_conclusao_em',
                  'TP_ANO_CONCLUIU': 'ano_conclusao_em',
                  'TP_ESCOLA': 'tipo_escola',
                  'TP_ENSINO': 'ensino',
                  'NO_MUNICIPIO_PROVA': 'municipio',
                  'SG_UF_PROVA': 'uf',
                  'TP_PRESENCA_CN': 'presenca_cn',
                  'TP_PRESENCA_CH': 'presenca_ch',
                  'TP_PRESENCA_LC': 'presenca_lc',
                  'TP_PRESENCA_MT': 'presenca_mt',
                  'NU_NOTA_CN': 'nota_cn',
                  'NU_NOTA_CH': 'nota_ch',
                  'NU_NOTA_LC': 'nota_lc',
                  'NU_NOTA_MT': 'nota_mt',
                  'TP_LINGUA': 'lingua_estrangeira',
                  'NU_NOTA_REDACAO': 'nota_redacao',
                  'Q001': 'escolaridade_pai',
                  'Q002': 'escolaridade_mae',
                  'Q005': 'pessoas_em_casa',  
                  'Q006': 'renda_mensal_bruta',
                  'Q022': 'possui_celular',
                  'Q024': 'possui_computador',
                  'Q025': 'possui_internet'}, inplace = True)

#Substituindo códigos
age_code = [x+1 for x in range(20)]
rj_dataset['faixa_etaria'] = rj_dataset['faixa_etaria'].replace(age_code, 
['Menor de 17 anos','17 anos', '18 anos', '19 anos', '20 anos','21 anos', '22 anos','23 anos','24 anos',
'25 anos',
'Entre 26 e 30 anos',
'Entre 31 e 35 anos',
'Entre 36 e 40 anos',
'Entre 41 e 45 anos',
'Entre 46 e 50 anos',
'Entre 51 e 55 anos',
'Entre 56 e 60 anos',
'Entre 61 e 65 anos',
'Entre 66 e 70 anos',
'Maior de 70 anos'])

sex_code = ['M', 'F']
rj_dataset['sexo'] = rj_dataset['sexo'].replace(sex_code, ['Masculino', 'Feminino'])

civil_code = list(range(5))
rj_dataset['estado_civil'] = rj_dataset['estado_civil'].replace(civil_code, ['Não informado',
'Solteiro(a)',
'Casado(a)/Mora com companheiro(a)',
'Divorciado(a)/Desquitado(a)/Separado(a)',
'Viúvo(a)'])

race_code = list(range(6))
rj_dataset['cor_raca'] = rj_dataset['cor_raca'].replace(race_code, ['Não declarado',
'Branca',
'Preta',
'Parda',
'Amarela',
'Indígena'])

conclusion_code = [x+1 for x in range(4)]
rj_dataset['status_conclusao_em'] = rj_dataset['status_conclusao_em'].replace(conclusion_code, ['Já concluí o Ensino Médio',
'Estou cursando e concluirei o Ensino Médio em 2020',
'Estou cursando e concluirei o Ensino Médio após 2020',
'Não concluí e não estou cursando o Ensino Médio'])

conclusion_year_code = list(range(15))
rj_dataset['ano_conclusao_em'] = rj_dataset['ano_conclusao_em'].replace(conclusion_year_code, ['Não informado',
'2019',
'2018',
'2017',
'2016',
'2015',
'2014',
'2013',
'2012',
'2011',
'2010',
'2009',
'2008',
'2007',
'Antes de 2007'])

school_code = [x+1 for x in range(4)]
rj_dataset['tipo_escola'] = rj_dataset['tipo_escola'].replace(school_code, ['Não Respondeu',
'Pública',
'Privada',
'Exterior'])

segment_code = [x+1 for x in range(3)]
rj_dataset['ensino'] = rj_dataset['ensino'].replace(segment_code, ['Ensino Regular',
'Educação Especial - Modalidade Substitutiva',
'Educação de Jovens e Adultos'])


presenca_code = list(range(3))
for col in ['presenca_cn', 'presenca_ch', 'presenca_lc', 'presenca_mt']:
  rj_dataset[col] = rj_dataset[col].replace(presenca_code, ['Faltou à prova',  'Presente na prova',  'Eliminado na prova'])

language_code = list(range(2))
rj_dataset['lingua_estrangeira'] = rj_dataset['lingua_estrangeira'].replace(language_code, ['Inglês', 'Espanhol'])

celplhone_code = ['A', 'B', 'C', 'D', 'E']
rj_dataset['possui_celular'] = rj_dataset['possui_celular'].replace(celplhone_code, ['Não',
'Sim, um',
'Sim, dois',
'Sim, três',
'Sim, quatro ou mais'])


pc_code = ['A', 'B', 'C', 'D', 'E']
rj_dataset['possui_computador'] = rj_dataset['possui_computador'].replace(pc_code, ['Não',
'Sim, um',
'Sim, dois',
'Sim, três',
'Sim, quatro ou mais'])


internet_code = ['A', 'B']
rj_dataset['possui_internet'] = rj_dataset['possui_internet'].replace(internet_code, ['Não', 'Sim'])


In [9]:
rj_dataset.head()

,incricao,faixa_etaria,sexo,estado_civil,cor_raca,status_conclusao_em,ano_conclusao_em,tipo_escola,ensino,municipio,...,nota_mt,lingua_estrangeira,nota_redacao,escolaridade_pai,escolaridade_mae,pessoas_em_casa,renda_mensal_bruta,possui_celular,possui_computador,possui_internet
33,200001699039,Entre 31 e 35 anos,Feminino,Solteiro(a),Preta,Já concluí o Ensino Médio,2008,Não Respondeu,NaN,Campos dos Goytacazes,...,NaN,Espanhol,NaN,A,B,1.0,B,"Sim, um","Sim, um",Não
67,200001104094,21 anos,Feminino,Solteiro(a),Preta,Já concluí o Ensino Médio,2016,Não Respondeu,NaN,Rio de Janeiro,...,NaN,Inglês,NaN,B,E,6.0,B,"Sim, três",Não,Sim
93,200002727241,19 anos,Masculino,Solteiro(a),Preta,Estou cursando e concluirei o Ensino Médio em ...,Não informado,Pública,Ensino Regular,Itaguaí,...,743.0,Inglês,920.0,E,E,4.0,F,"Sim, três","Sim, dois",Sim
130,200002186164,17 anos,Feminino,Solteiro(a),Parda,Estou cursando e concluirei o Ensino Médio em ...,Não informado,Pública,Ensino Regular,Rio de Janeiro,...,538.6,Espanhol,560.0,C,C,2.0,B,"Sim, dois","Sim, um",Sim
137,200003838320,22 anos,Feminino,Solteiro(a),Parda,Já concluí o Ensino Médio,2015,Não Respondeu,NaN,Duque de Caxias,...,411.8,Espanhol,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Agregando presença:
# Alunos que faltaram ou foram eliminados em alguma prova serão classificados como 'Eliminado'
# Alunos presentes em todas as provas serão classificados como 'Presente'
status_geral = []
for i, row in rj_dataset.iterrows():
  if (row['presenca_cn'] == 'Presente na prova' and row['presenca_cn'] == 'Presente na prova' and row['presenca_lc'] == 'Presente na prova' and row['presenca_mt'] == 'Presente na prova'):
    status_geral.append('Presente')
  else:
    status_geral.append('Eliminado')

rj_dataset['status_geral'] = status_geral

# Removendo colunas induviduais de presenca
rj_dataset.drop(columns=['presenca_cn', 'presenca_ch', 'presenca_lc', 'presenca_mt'], inplace = True)

In [11]:
# Verificando notas nulas para Presentes
rj_dataset.query("status_geral == 'Presentes'")[['nota_ch', 'nota_cn', 'nota_lc', 'nota_mt','nota_redacao']].isnull().sum()

nota_ch         0.0
nota_cn         0.0
nota_lc         0.0
nota_mt         0.0
nota_redacao    0.0
dtype: float64

In [12]:
# Verificando notas nulas para Eliminados
rj_dataset.query("status_geral == 'Eliminado'")[['nota_ch', 'nota_cn', 'nota_lc', 'nota_mt','nota_redacao']].isnull().sum()

nota_ch         191994
nota_cn         201312
nota_lc         191994
nota_mt         201312
nota_redacao    191994
dtype: int64

In [13]:
rj_dataset.head()

,incricao,faixa_etaria,sexo,estado_civil,cor_raca,status_conclusao_em,ano_conclusao_em,tipo_escola,ensino,municipio,...,lingua_estrangeira,nota_redacao,escolaridade_pai,escolaridade_mae,pessoas_em_casa,renda_mensal_bruta,possui_celular,possui_computador,possui_internet,status_geral
33,200001699039,Entre 31 e 35 anos,Feminino,Solteiro(a),Preta,Já concluí o Ensino Médio,2008,Não Respondeu,NaN,Campos dos Goytacazes,...,Espanhol,NaN,A,B,1.0,B,"Sim, um","Sim, um",Não,Eliminado
67,200001104094,21 anos,Feminino,Solteiro(a),Preta,Já concluí o Ensino Médio,2016,Não Respondeu,NaN,Rio de Janeiro,...,Inglês,NaN,B,E,6.0,B,"Sim, três",Não,Sim,Eliminado
93,200002727241,19 anos,Masculino,Solteiro(a),Preta,Estou cursando e concluirei o Ensino Médio em ...,Não informado,Pública,Ensino Regular,Itaguaí,...,Inglês,920.0,E,E,4.0,F,"Sim, três","Sim, dois",Sim,Presente
130,200002186164,17 anos,Feminino,Solteiro(a),Parda,Estou cursando e concluirei o Ensino Médio em ...,Não informado,Pública,Ensino Regular,Rio de Janeiro,...,Espanhol,560.0,C,C,2.0,B,"Sim, dois","Sim, um",Sim,Presente
137,200003838320,22 anos,Feminino,Solteiro(a),Parda,Já concluí o Ensino Médio,2015,Não Respondeu,NaN,Duque de Caxias,...,Espanhol,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Presente


In [14]:
# Tratando dados nulos
rj_dataset.isnull().sum()

incricao                    0
faixa_etaria                0
sexo                        0
estado_civil                0
cor_raca                    0
status_conclusao_em         0
ano_conclusao_em            0
tipo_escola                 0
ensino                 268165
municipio                   0
uf                          0
nota_cn                201312
nota_ch                191994
nota_lc                191994
nota_mt                201312
lingua_estrangeira          0
nota_redacao           191994
escolaridade_pai         6792
escolaridade_mae         6792
pessoas_em_casa          6792
renda_mensal_bruta       6792
possui_celular           6792
possui_computador        6792
possui_internet          6792
status_geral                0
dtype: int64

In [15]:
#Substituindo valores nulos para cada uma das colunas que tem valores nulos (ou NaN)

#Numéricas
for col in ['nota_ch','nota_cn', 'nota_lc', 'nota_mt', 'nota_redacao']:
  rj_dataset[col].fillna(0, inplace = True)

#Categóricas
for col in ['ensino','escolaridade_pai', 'escolaridade_mae', 'pessoas_em_casa', 'renda_mensal_bruta', 'possui_celular', 'possui_computador', 'possui_internet']:
  rj_dataset[col].fillna('Não informado', inplace = True)

#Verificação
rj_dataset.isnull().sum()

incricao               0
faixa_etaria           0
sexo                   0
estado_civil           0
cor_raca               0
status_conclusao_em    0
ano_conclusao_em       0
tipo_escola            0
ensino                 0
municipio              0
uf                     0
nota_cn                0
nota_ch                0
nota_lc                0
nota_mt                0
lingua_estrangeira     0
nota_redacao           0
escolaridade_pai       0
escolaridade_mae       0
pessoas_em_casa        0
renda_mensal_bruta     0
possui_celular         0
possui_computador      0
possui_internet        0
status_geral           0
dtype: int64

## Carregamento de dados
Novo CSV com o dataset tratado

In [16]:
print(f'O dataset final que será utilizado para as análises tem {rj_dataset.shape[0]} registros e {rj_dataset.shape[1]} variáveis')

O dataset final que será utilizado para as análises tem 356206 registros e 25 variáveis


In [17]:
rj_dataset.to_csv('/content/drive/MyDrive/Projetos/Python/analise_socio_economica_enem_2020/data/enem2020_rj_dataset.csv', sep = ',', encoding='iso-8859-1', index = False)

# Análises de dados
